In [11]:
# imports
import requests
import os
from lxml import html
from htmldom import htmldom
from PyPDF4 import PdfFileMerger, PdfFileReader

In [20]:
# setup
directory = "moderate" # leave blank for current
archive_url = "https://www.killersudokuonline.com/archives-2021.html"
completed = [5493,5498,5499,5506,5512,5513,5519,5520,5526,5527,5533] # don't download these
combined_directory = 'moderate_combined'
solution_links = 'solution_links.html'

In [3]:
# get permalinks to moderate puzzle pages
dom = htmldom.HtmlDom(archive_url)
dom = dom.createDom()

p = dom.find("a[name=KillerSudoku]").children("table")

moderate = p[2] #easier would be 0, easy 1, etc
# print(moderate.html())

moderate_links= moderate.find("a")
# print(moderate_links[1].html())

moderate_dict={}
for i in moderate_links:
    name = int(i.text()[10:])
    permalink = "https://www.killersudokuonline.com/"+i.attr("href")
    moderate_dict[name]=dict(url = permalink)
# print(moderate_dict)

# remove completed
for num in completed:
    if num in moderate_dict.keys():
        moderate_dict.pop(num)

In [4]:
# get links for puzzles and solutions
for i in moderate_dict.keys():
    dom = htmldom.HtmlDom(moderate_dict[i]["url"])
    dom = dom.createDom()

    k = dom.find("a[name=kdaily]").children("table")
    # print(k.html())
    links = k.find("a")
    # print(links[0].html())

    moderate_dict[i]['puzzle'] = "https://www.killersudokuonline.com" + links[0].attr("href")
    # print(puzzle)
    moderate_dict[i]['solution'] = "https://www.killersudokuonline.com" + links[2].attr("href")
    # print(solution)

In [7]:
# generate list of save paths
if directory == "": directory=str(os.getcwd())
for i in moderate_dict:
    newTitle = str(i)+".pdf"
    thisPath = directory+"\\"+newTitle
    moderate_dict[i]['filepath'] = thisPath
# print(paths)

# # access and download puzzles
for i in moderate_dict:
    r = requests.get(moderate_dict[i]['puzzle']) # request that webpage
    newFile = open(moderate_dict[i]['filepath'], 'wb+')
    newFile.write(r.content) # create a file, and write to it
    newFile.close()

In [10]:
# combine 2 pdfs for easy double sided printing
files = os.listdir(directory)[1:]
for pair in range(0, (len(files)),2):
    print(files[pair], files[pair+1])
    name = combined_directory+'\\'+files[pair][:-4]+'_'+files[pair+1]
    # print(name)
    
    myfiles = [open(directory+'\\'+files[pair],"rb"), open(directory+'\\'+files[pair+1], "rb")]
    merger = PdfFileMerger(open(name, "wb"))
    merger.append(fileobj=myfiles[0])
    merger.append(fileobj=myfiles[1])
    
    merger.write(fileobj=name)
    merger.close()

5534.pdf 5540.pdf
5541.pdf 5547.pdf
5548.pdf 5549.pdf
5554.pdf 5555.pdf
5556.pdf 5561.pdf
5562.pdf 5568.pdf
5569.pdf 5570.pdf
5575.pdf 5576.pdf
5581.pdf 5582.pdf
5583.pdf 5589.pdf
5590.pdf 5596.pdf
5597.pdf 5603.pdf
5604.pdf 5605.pdf
5610.pdf 5611.pdf
5612.pdf 5617.pdf
5618.pdf 5624.pdf
5625.pdf 5626.pdf
5632.pdf 5638.pdf
5639.pdf 5645.pdf
5646.pdf 5652.pdf
5653.pdf 5659.pdf
5660.pdf 5665.pdf
5666.pdf 5667.pdf
5668.pdf 5673.pdf
5674.pdf 5680.pdf
5681.pdf 5688.pdf
5693.pdf 5694.pdf
5695.pdf 5702.pdf
5708.pdf 5709.pdf
5710.pdf 5715.pdf
5716.pdf 5722.pdf
5723.pdf 5724.pdf
5730.pdf 5731.pdf
5735.pdf 5736.pdf
5737.pdf 5738.pdf
5743.pdf 5744.pdf
5750.pdf 5751.pdf
5757.pdf 5758.pdf
5765.pdf 5766.pdf
5771.pdf 5772.pdf
5777.pdf 5778.pdf
5779.pdf 5785.pdf
5786.pdf 5793.pdf
5800.pdf 5801.pdf
5806.pdf 5807.pdf
5813.pdf 5814.pdf
5820.pdf 5821.pdf
5822.pdf 5826.pdf
5827.pdf 5828.pdf
5835.pdf 5841.pdf
5842.pdf 5848.pdf
5849.pdf 5856.pdf


In [24]:
# create a webpage with links to solutions
txt=""
filename = combined_directory + '\\' + solution_links
f = open(filename,"w+")
f.write("<!DOCTYPE html> <html><body>")
for i in moderate_dict:
    link = '<a href = "' +  moderate_dict[i]['solution'] + '">' + str(i) + '</a><br><br>'
    f.write(link)
f.write("</body></html>")
f.close()